# Open datasets

# Acquiring Data from open repositories

A crucial step in the work of a computational biologist is not only to analyse data, but acquiring datasets to analyse as well as toy datasets to test out computational methods and algorithms. The internet is full of such open datasets. Sometimes you have to sign up and make a user to get authentication, especially for medical data. This can sometimes be time consuming, so here we will deal with easy access resources, mostly of modest size. Multiple python libraries provide a `dataset` module which makes the effort to fetch online data extremely seamless, with little requirement for preprocessing.


### Goal of the notebook

Here you will get familiar with some ways to fetch datasets from online. We do some data exploration on the data just for illustration, but the methods will be covered later.


# Useful resources and links

When playing around with algorithms, it can be practical to use relatively small datasets. A good example is the `datasets` submodule of `scikit-learn`. `Nilearn` (library for neuroimaging) also provides a collection of neuroimaging datasets. Many datasets can also be acquired through the competition website [Kaggle](https://www.kaggle.com), in which they describe how to access the data.


### Links
- [OpenML](https://www.openml.org/search?type=data)
- [Nilearn datasets](https://nilearn.github.io/modules/reference.html#module-nilearn.datasets)
- [Sklearn datasets](https://scikit-learn.org/stable/modules/classes.html?highlight=datasets#module-sklearn.datasets)
- [Kaggle](https://www.kaggle.com/datasets)
- [MEDNIST]

-  [**Awesomedata**](https://github.com/awesomedata/awesome-public-datasets)

 - We strongly recommend to check out the Awesomedata lists of public datasets, covering topics such as [biology/medicine](https://github.com/awesomedata/awesome-public-datasets#biology) and [neuroscience](https://github.com/awesomedata/awesome-public-datasets#neuroscience)

- [Papers with code](https://paperswithcode.com)

- [SNAP](https://snap.stanford.edu/data/)
  - Stanford Large Network Dataset Collection  
- [Open Graph Benchmark (OGB)](https://github.com/snap-stanford/ogb)
  - Network datasets
- [Open Neuro](https://openneuro.org/)
- [Open fMRI](https://openfmri.org/dataset/)

In [ ]:
# import basic libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

We start with scikit-learn's datasets for testing out Machine Learning (ML) algorithms. Visit [here](https://scikit-learn.org/stable/modules/classes.html?highlight=datasets#module-sklearn.datasets) for an overview of the datasets.

In [1]:
#import multiple datasets from sklearn package. These are functions we can call to get said dataset
from sklearn.datasets import fetch_olivetti_faces, fetch_20newsgroups, load_breast_cancer, load_diabetes, load_digits, load_iris

# Hand written digits

Load the MNIST dataset that consists of images of hand written digits

In [ ]:
# load data (pixels) to X and target (the number the image represents) to y
X,y = load_digits(return_X_y=True)

In [ ]:
y.shape

In [ ]:
X.shape #1797 images, 64 pixels per image

In [ ]:
# values in array are grey values of the image (0=black, 15=white)
X[0]

In [ ]:
# y has the labels, i.e. pixels above represent the number in same index in y
y[0]

**Plot the image**

In [ ]:
# plot first image. We need to reshape the 64 pixels into [8,8] array
plt.imshow(X[0].reshape(8,8), cmap='gray')
plt.show()

<div class='alert alert-warning'>
<h4> Exercise 1.</h4>  Make a function `plot()` taking an argument (k) to visualize the k'th sample. 
It is currently flattened, you will need to reshape it. Use `plt.imshow` for plotting and add title of the number the image presents. 

In [ ]:
# Ex1


In [ ]:
# %load solutions/ex2_1.py

In [ ]:
# test your solution
plot(15); plot(450)

# Olivetti face data

Dataset of 40 subjects faces with varying facial expressions, facial details and lighting

In [ ]:
faces = fetch_olivetti_faces()

<div class='alert alert-warning'>
<h4>Exercise 2. </h4> Inspect the dataset. How many classes are there? How many samples per class? Also, plot some examples. What do the classes represent? 
</div>

*Hint: Write `faces.` and press tab to see what attributes dataset has.*    

In [ ]:
# Ex2


In [ ]:
# %load solutions/ex2_2.py

Once you have made yourself familiar with the dataset we can do some data exploration with unsupervised methods, like below. The next few lines of code are simply for illustration, don't worry about the code (we will cover unsupervised methods in submodule F).

In [ ]:
from sklearn.decomposition import randomized_svd

In [ ]:
X = faces.data
y = faces.target

In [ ]:
n_dim = 3
u, s, v = randomized_svd(X, n_dim)

Now we have factorized the images into their constituent parts. The code below displays the various components isolated one by one.

In [ ]:
# don't worry about this code
def show_ims(ims):
    fig = plt.figure(figsize=(16,10))
    idxs = [0,1,2, 11,12,13, 40,41,42, 101,101,103]
    for i,k in enumerate(idxs):
        ax=fig.add_subplot(3,4,i+1)
        ax.imshow(ims[k])
        ax.set_title(f"target={y[k]}")

In [ ]:
for i in range(n_dim):
    my_s = np.zeros(s.shape[0])
    my_s[i] = s[i]
    recon = u@np.diag(my_s)@v
    recon = recon.reshape(400,64,64)
    show_ims(recon)

The components highlight different features of the image. Are you able to see what features the components represent? It at least looks like the second component signifies the lightning  (the light direction), the third highlights eyebrows and facial chin shape.


# Fetching an OpenML dataset

Here we will look at [OpenML](https://www.openml.org/) - a repository of open datasets free to explore data and test methods.

We need to pass in an ID to access, as follows:

In [ ]:
from sklearn.datasets import fetch_openml

OpenML contains all sorts of datatypes. By browsing the website we found a electroencephalography (EEG) dataset to explore: 

In [ ]:
data_id = 1471 #this was found by browsing OpenML
dataset = fetch_openml(data_id=data_id, as_frame=True)

In [ ]:
dir(dataset)

In [ ]:
dataset.url

In [ ]:
type(dataset)

In [ ]:
print(dataset.DESCR)

In [ ]:
original_names=['AF3', 'F7', 'F3', 'FC5', 'T7', 'P', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']

In [ ]:
dataset.feature_names

In [ ]:
df = dataset.frame

In [ ]:
df.head()

In [ ]:
df.shape[0] / 117
# 128 frames per second

In [ ]:
df.dtypes

In [ ]:
df.plot()

From the plot we can quickly identify a bunch of huge outliers, making the plot look completely uselss. We assume these are artifacts, and remove them.

In [ ]:
df2 = df.iloc[:,:-1].clip(upper=6000) #Elements above the threshold will be changed to match the threshold value.
df2.plot()

Now we see better what is going on. Lets just remove the frames corresponding to those outliers

In [ ]:
# which are the frames that are above 5000
frames = np.nonzero(np.any(df.iloc[:,:-1].values>5000, axis=1))[0] 
frames

In [ ]:
# remove those frames
df.drop(index=frames, inplace=True)

In [ ]:
# plotting without outliers
df.plot(figsize=(16,8))
plt.legend(labels=original_names)